In [ ]:
import mysql.connector
cnx = mysql.connector.connect(user='root', password='root',
                              host='127.0.0.1',
                              database='usTreasury')

TABLES = {}
TABLES['usTreasury'] = (
    "CREATE TABLE `usTreasury` ("
    "  `date` varchar(11) NOT NULL,"
    "  `1 month` varchar(5) NOT NULL,"
    "  `3 month` varchar(5) NOT NULL,"
    "  `6 month` varchar(5) NOT NULL,"
    "  `1 year` varchar(5) NOT NULL,"
    "  `2 year` varchar(5) NOT NULL,"
    "  `3 year` varchar(5) NOT NULL,"
    "  `5 year` varchar(5) NOT NULL,"
    "  `7 year` varchar(5) NOT NULL,"
    "  `10 year` varchar(5) NOT NULL,"
    "  `20 year` varchar(5) NOT NULL,"
    "  `30 year` varchar(5) NOT NULL,"
    "  PRIMARY KEY (`date`)"
    ") ENGINE=InnoDB")
CREATE TABLES 
cnx.close()

In [ ]:
import MySQLdb
db = MySQLdb.connect("localhost","root","root","usTreasury" )
cursor = db.cursor()
# cursor.execute('select * from usTreasury')
# tablerows=cursor.fetchall()

sql = """CREATE TABLE usTreasury (
         date  DATE NOT NULL,
        1_month CHAR(5), 
        3_month CHAR(5),
        6_month CHAR(5), 
        1_year CHAR(5), 
        2_year CHAR(5),
        3_year CHAR(5), 
        5_year CHAR(5), 
        7_year CHAR(5), 
        10_year CHAR(5), 
        20_year CHAR(5), 
        30_year CHAR(5)
         )"""
cursor.execute(sql)



In [ ]:
import MySQLdb


db = MySQLdb.connect("localhost","root","root","usTreasury" )


cursor = db.cursor()


a = [u'12/31/14', u'0.03', u'0.04', u'0.12', u'0.25', u'0.67', u'1.10', u'1.65', u'1.97', u'2.17', u'2.47', u'2.75']

sql = """INSERT INTO usTreasury(date,
         1_month, 3_month, 6_month, 1_year,2_year,3_year,5_year,7_year,10_year,20_year,30_year)
         VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
try:
   
   cursor.execute(sql,a)
   
   db.commit()
except:
   
   db.rollback()

    
sql = "SELECT * from usTreasury where date = (SELECT max(date) FROM usTreasury)"
cursor.execute(sql)
results = cursor.fetchone()
print results


db.close()

In [ ]:
# -------------------------------------------  
# Python MySQLdb 循环插入execute与批量插入executemany性能分析  
# 插入数据量：10000条  
# 每条字段：username, salt, pwd  
# Author : Lrg  
# -------------------------------------------  
# encoding = utf-8  
import MySQLdb  
import xlrd  
import time  
import sys  
reload(sys)  
sys.setdefaultencoding("utf-8")  
  
# 从users.xls文件获取10000条用户数据  
# 该文件由create_users.py生成  
def get_table():  
    FILE_NAME = 'users.xls'  
    data = xlrd.open_workbook(FILE_NAME)  
    table = data.sheets()[0]  
    return table  
  
# 循环插入execute     
def insert_by_loop(table):  
    nrows = table.nrows  
    for i in xrange(1,nrows):  
        param=[]  
        try:  
            sql = 'INSERT INTO user values(%s,%s,%s)'  
            # 第一列username，第二列salt，第三列pwd  
            print 'Insert: ',table.cell(i, 0).value, table.cell(i, 1).value, table.cell(i, 2).value  
            param = (table.cell(i, 0).value, table.cell(i, 1).value, table.cell(i, 2).value)  
            # 单条插入  
            cur.execute(sql, param)  
            conn.commit()  
        except Exception as e:  
            print e  
            conn.rollback()  
    print '[insert_by_loop execute] total:',nrows-1  
  
# 批量插入executemany  
def insert_by_many(table):  
    nrows = table.nrows  
    param=[]  
    for i in xrange(1,nrows):  
        # 第一列username，第二列salt，第三列pwd  
        param.append([table.cell(i, 0).value, table.cell(i, 1).value, table.cell(i, 2).value])  
    try:  
        sql = 'INSERT INTO user values(%s,%s,%s)'  
        # 批量插入  
        cur.executemany(sql, param)  
        conn.commit()  
    except Exception as e:  
        print e  
        conn.rollback()   
    print '[insert_by_many executemany] total:',nrows-1   
  
  
# 连接数据库  
conn = MySQLdb.connect(host="127.0.0.1", port=3306, user="lrg", passwd="lrg", db="pythontest")  
cur = conn.cursor()  
  
# 新建数据库  
cur.execute('DROP TABLE IF EXISTS user')  
sql = """CREATE TABLE user( 
        username CHAR(255) NOT NULL, 
        salt CHAR(255), 
        pwd CHAR(255) 
        )"""  
cur.execute(sql)  
  
# 从excel文件获取数据  
table = get_table()  
  
# 使用循环插入  
start = time.clock()  
insert_by_loop(table)  
end = time.clock()  
print '[insert_by_loop execute] Time Usage:',end-start  
  
# 使用批量插入  
start = time.clock()  
insert_by_many(table)  
end = time.clock()  
print '[insert_by_many executemany] Time Usage:',end-start  
  
# 释放数据连接  
if cur:  
    cur.close()  
if conn:  
    conn.close()  